# LLM Book Processing Notebook

In [ ]:
!pip install openai python-dotenv requests

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("API key not found. Set OPENAI_API_KEY in .env")


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)


In [ ]:
def genereer_samenvatting(titel, eisen):
    prompt = f"""
Geef een objectieve, informatieve samenvatting van het boek met de titel:
'{titel}'.

Eisen:
- {eisen}
Gebruik geen spoilers buiten de hoofdlijn en geef de kern van het verhaal weer.
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


In [ ]:
titel = "The Silent Patient"
eisen = "maximaal 150 woorden, objectief, geen spoilers."
genereer_samenvatting(titel, eisen)


In [ ]:
import json
import re

def verrijk_semantiek(titel: str):
    prompt = f"""
Je bent een JSON-generator. Geef *alleen* valide JSON terug en niets anders.

Analyseer het boek '{titel}' en geef:

- thema's
- motieven
- toon/sfeer
- schrijfstijl
- karaktertypen
- narratieve structuur

Formaat (verplicht):

{{
    "themes": ["..."],
    "motifs": ["..."],
    "tone": "...",
    "writing_style": "...",
    "character_types": ["..."],
    "narrative_structure": "..."
}}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Je output is ALTIJD pure JSON. Geen tekst eromheen."},
            {"role": "user", "content": prompt}
        ]
    )

    raw = response.choices[0].message.content.strip()

    # 1. Probeer direct JSON
    try:
        return json.loads(raw)
    except:
        pass

    # 2. Zo niet: extracteer JSON-blok uit tekst
    try:
        json_text = re.search(r"\{.*\}", raw, re.DOTALL).group(0)
        return json.loads(json_text)
    except:
        print("AI-output kon niet worden geparsed:")
        print(raw)
        raise


In [ ]:
verrijk_semantiek("The Silent Patient")

In [ ]:
def verwerk_boek_json(titel: str, eisen: str = None):
    data = verwerk_boek(titel, eisen)
    return json.dumps(data, indent=4, ensure_ascii=False)


In [ ]:
json_blob = verwerk_boek_json("The Silent Patient")
print(json_blob)